In [66]:
from ultralytics import YOLO
import numpy as np
import cv2
from mss import mss
from PIL import Image, ImageDraw
from collections import defaultdict
import time
import torch

model = YOLO("runs/detect/train8/weights/best.pt")

In [62]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

model.to(device)

In [72]:
video_path = "tenis2.mp4"

cap = cv2.VideoCapture(video_path)

tracked_points = []

while cap.isOpened():
    success, frame = cap.read()

    if success:
        results = model.track(frame, persist=True, conf=0.50)        
        annotated_frame = results[0].plot()

        if results[0].boxes.shape[0] > 0:
            first_box = results[0].boxes.data[0]

            # Extract coordinates
            x1 = first_box[0]
            y1 = first_box[1]
            x2 = first_box[2]
            y2 = first_box[3]

            center_x = int((x1 + x2) / 2)
            center_y = int((y1 + y2) / 2)

            tracked_points.append(((center_x, center_y), time.time()))
        current_time = time.time()
        tracked_points = [(pt, t) for (pt, t) in tracked_points if current_time - t < 3]
    

        for i in range(1, len(tracked_points)):
            if tracked_points[i - 1] is None or tracked_points[i] is None:
                continue
            pt1, _ = tracked_points[i - 1]
            pt2, _ = tracked_points[i]
            cv2.line(annotated_frame, pt1, pt2, (0, 255, 0), 2)



        cv2.imshow("YOLOv8 Tracking", annotated_frame)

        if cv2.waitKey(1) & 0xFF == ord("q") or cv2.waitKey(1) & 0xFF == ord("Q"):
            break
    else:
        break

cap.release()
cv2.destroyAllWindows()


0: 384x640 (no detections), 126.6ms
Speed: 3.0ms preprocess, 126.6ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 106.6ms
Speed: 2.0ms preprocess, 106.6ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 12.0ms
Speed: 1.0ms preprocess, 12.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 12.0ms
Speed: 2.0ms preprocess, 12.0ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 12.0ms
Speed: 2.0ms preprocess, 12.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 13.0ms
Speed: 1.0ms preprocess, 13.0ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 13.5ms
Speed: 1.0ms preprocess, 13.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 17.0ms
Speed: 1.0ms preprocess, 17.0

In [59]:
results[0].boxes.data[0][1]

tensor(71.7820)